In [ ]:
from  transformers  import XLMRobertaModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
print('Model and Tokenizer Loaded.')

In [ ]:
import main as m
def build_most_freq_tokens(corpus_file ,  token_count): 
    dico = {}
    total_tokens = 0 
    with open(corpus_file, 'r' , encoding='utf8') as f: 
        for i, line in enumerate(f):
            tokens = list(filter(None, re.split(';|\,|\s|\.|\?|\!|\؟' , line.strip())))
            for token in tokens: 
                if token in dico:
                    dico[token] += 1 
                else: 
                    dico[token] = 1 
                total_tokens += 1 
    srt = sorted(dico.items() , key= lambda i: i[1] , reverse=True) 
    ret = []
    selected_tokens = 0 
    for i in range(token_count):
        selected_tokens += srt[i][1] 
        ret.append(srt[i][0])
    print ('Total token {}  we selected {}  which is {} percent'.format(total_tokens, selected_tokens,  selected_tokens / total_tokens * 100))
    return ret 
m.extract_dictionary(build_most_freq_tokens('../data/common/blogs-10M.txt' ,  30000) , '../data/common/fa-top.txt') 
m.extract_dictionary(build_most_freq_tokens('../data/common/wiki-en-30K.txt' ,  30000) , '../data/common/en-top.txt') 


In [ ]:
import numpy as np 

def create_mean_vectors(top_word_file , corpus_file , output_file , model , tokenizer , line_count = 10000 ):
    dico = {}
    dico_count = {} 
    with open(top_word_file, 'r' , encoding='utf8') as f: 
        for i, line in enumerate(f):
            dico_count[line.strip()] =  0
            dico[line.strip()] = np.zeros([768])


    with open(corpus_file, 'r' , encoding='utf8') as f: 
        found = 0 
        not_found = 0 
        for i, line in enumerate(f):
            if i > line_count:
                break

            tokens = tokenizer(line)
            lst = tokenizer.convert_ids_to_tokens(tokens['input_ids']) 
            
            input_ids = torch.tensor(tokens['input_ids']).unsqueeze(0)  
            outputs = model(input_ids)

            #print ('Line:',  line )
            #print('Tokens:' , lst)

            for ind , item in enumerate(lst): 
                if item == '<s>' or item == '</s>':
                    continue 
                else: 
                    item = item.replace('▁' , '') 
                    #print(item)

                    if item in dico_count: 
                        found += 1 
                        dico_count[item] += 1 
                        dico[item] += outputs[0][0, ind , : ].detach().numpy() 
                    else:
                        not_found += 1 
            if i % 100 == 1:
                print('Found {} not found {}  Success Rate: {}'.format(found, not_found ,  found / (found + not_found)))

    non_zero = 0 
    for t, c in dico_count.items():
        if c > 0: 
            non_zero += 1 
    with open(output_file , 'w' , encoding='utf8') as out: 
        out.write(str(non_zero) +  ' ' + str(768) + '\n') 
        for token in dico: 
            if dico_count[token] > 0:
                out.write(token + ' ')
                for x in (dico[token] / dico_count[token]).tolist(): 
                    out.write('{:.4f} '.format(x)) 
                out.write('\n') 
    



create_mean_vectors('../data/common/fa-top.txt' , '../data/common/blogs-100K.txt' , '../data/common/fa.vec' , model , tokenizer , 5)